In [1]:
#全局变量
hub_token = open('/root/hub_token.txt').read().strip()
repo_id = 'lansinuote/nlp.4.summarization'
push_to_hub = True

In [2]:
from transformers import AutoTokenizer

#加载分词器
tokenizer = AutoTokenizer.from_pretrained('t5-small')

print(tokenizer)

#编码试算
print(
    tokenizer.batch_encode_plus(
        ['Hello, this one sentence!', 'This is another sentence.']))

#label的编码方式,但是试验结果是和input的编码方式一样,没有区别
with tokenizer.as_target_tokenizer():
    print(
        tokenizer.batch_encode_plus(
            ['Hello, this one sentence!', 'This is another sentence.']))

T5TokenizerFast(name_or_path='t5-small', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_i

/root/anaconda3/envs/pt39/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
/root/anaconda3/envs/pt39/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by usin

In [3]:
from datasets import load_dataset


def get_dataset():
    #加载数据集
    dataset = load_dataset('xsum')

    #采样,数据量太大了跑不动
    dataset['train'] = dataset['train'].shuffle(1).select(range(20000))
    dataset['validation'] = dataset['validation'].shuffle(1).select(range(100))
    dataset['test'] = dataset['test'].shuffle(1).select(range(100))

    print(dataset, dataset['train'][0])

    #数据预处理函数,分词
    def f(examples):
        #编码input
        data = tokenizer.batch_encode_plus(
            #在输入的前面加summarize前缀,这个是h5模型识别任务类型的标识
            ['summarize: ' + i for i in examples['document']],
            max_length=1024,
            truncation=True,
        )

        #编码label
        with tokenizer.as_target_tokenizer():
            data['labels'] = tokenizer.batch_encode_plus(
                examples['summary'],
                max_length=128,
                truncation=True,
            )['input_ids']

        return data

    dataset = dataset.map(function=f,
                          batched=True,
                          batch_size=1000,
                          num_proc=4,
                          remove_columns=['document', 'summary', 'id'])

    return dataset


if push_to_hub:
    dataset = get_dataset()
    dataset.push_to_hub(repo_id=repo_id, token=hub_token)

#直接使用我处理好的数据集
dataset = load_dataset(path=repo_id)

print(dataset, dataset['train'][0])

Found cached dataset xsum (/root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71/cache-60c989397024aa38.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71/cache-c55e23852740fe4a.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71/cache-7363f4d927e806d4.arrow


DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 100
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 100
    })
}) {'document': "Clay, who has agreed a two-year deal, made 39 appearances for Scottish Premiership club Motherwell last season after joining them in June 2016.\nThe 25-year-old had spent the two previous seasons with Grimsby, playing 74 National League games.\nClay is Leyton Orient's ninth signing since being relegated from League Two last season.", 'summary': 'National League side Leyton Orient have signed Motherwell midfielder Craig Clay on a free transfer.', 'id': '40635923'}
 

Loading cached processed dataset at /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71/cache-8781f1973c983dd2.arrow


...

Generating validation split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/lansinuote___parquet/lansinuote--nlp.4.summarization-e0d3b57ef58076bd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
}) {'input_ids': [21603, 10, 20988, 6, 113, 65, 4686, 3, 9, 192, 18, 1201, 1154, 6, 263, 6352, 3179, 7, 21, 12580, 6552, 2009, 1886, 8007, 2091, 336, 774, 227, 6109, 135, 16, 1515, 4619, 37, 944, 18, 1201, 18, 1490, 141, 1869, 8, 192, 1767, 9385, 28, 23427, 7, 969, 6, 1556, 3, 4581, 868, 3815, 1031, 5, 20988, 19, 312, 21220, 3, 16495, 31, 7, 24651, 8097, 437, 271, 3, 60, 8791, 26, 45, 3815, 2759, 336, 774, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [4]:
#这个函数和下面这个工具类等价,但我也是模仿实现的,不确定有没有出入
#from transformers import DataCollatorForSeq2Seq
#DataCollatorForSeq2Seq(tokenizer, model=model)

import torch


#数据批处理函数
def collate_fn(data):
    #求最长的label
    max_length = max([len(i['labels']) for i in data])

    #把所有的label都补pad到最长
    for i in data:
        pads = [-100] * (max_length - len(i['labels']))
        i['labels'] = i['labels'] + pads

    #把多个数据整合成一个tensor
    data = tokenizer.pad(
        encoded_inputs=data,
        padding=True,
        max_length=None,
        pad_to_multiple_of=None,
        return_tensors='pt',
    )

    #定义decoder_input_ids
    data['decoder_input_ids'] = torch.zeros_like(data['labels'],
                                                 dtype=torch.long)
    data['decoder_input_ids'][:, 1:] = data['labels'][:, :-1]
    data['decoder_input_ids'][data['decoder_input_ids'] == -100] = 0

    return data


data = [{
    'input_ids': [21603, 10, 37, 3719, 13],
    'attention_mask': [1, 1, 1, 1, 1],
    'labels': [10455, 120, 80]
}, {
    'input_ids': [21603, 10, 7086, 8408, 563],
    'attention_mask': [1, 1, 1, 1, 1],
    'labels': [301, 53, 4074, 1669]
}]

collate_fn(data)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[21603,    10,    37,  3719,    13],
        [21603,    10,  7086,  8408,   563]]), 'attention_mask': tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]]), 'labels': tensor([[10455,   120,    80,  -100],
        [  301,    53,  4074,  1669]]), 'decoder_input_ids': tensor([[    0, 10455,   120,    80],
        [    0,   301,    53,  4074]])}

In [5]:
#数据加载器
loader = torch.utils.data.DataLoader(
    dataset=dataset['train'],
    batch_size=8,
    collate_fn=collate_fn,
    shuffle=True,
    drop_last=True,
)

for i, data in enumerate(loader):
    break

for k, v in data.items():
    print(k, v.shape)

len(loader)

input_ids torch.Size([8, 1024])
attention_mask torch.Size([8, 1024])
labels torch.Size([8, 49])
decoder_input_ids torch.Size([8, 49])


2500

In [6]:
from transformers import AutoModelForSeq2SeqLM, T5Model, PreTrainedModel, PretrainedConfig

#加载模型
#model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')


#定义下游任务模型
class Model(PreTrainedModel):
    config_class = PretrainedConfig

    def __init__(self,config):
        super().__init__(config)
        self.pretrained = T5Model.from_pretrained('t5-small')

        #本来应该写tokenizer.vocab_size=32100就可以了。
        #但是预训练模型里的参数是32128，所以为了方便就直接使用这个尺寸了
        self.fc = torch.nn.Linear(512, 32128, bias=False)

        #加载预训练模型的参数
        parameters = AutoModelForSeq2SeqLM.from_pretrained('t5-small')
        self.fc.load_state_dict(parameters.lm_head.state_dict())

        self.criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

    def forward(self, input_ids, attention_mask, labels, decoder_input_ids):

        logits = self.pretrained.encoder(input_ids=input_ids,
                                         attention_mask=attention_mask)

        logits = logits.last_hidden_state

        logits = self.pretrained.decoder(
            input_ids=decoder_input_ids,
            encoder_hidden_states=logits,
            encoder_attention_mask=attention_mask,
        )

        logits = logits.last_hidden_state

        logits = logits * (512**-0.5)

        logits = self.fc(logits)

        loss = self.criterion(logits.reshape(-1, 32128), labels.reshape(-1))

        return {'loss': loss, 'logits': logits}


model = Model(PretrainedConfig())

#统计参数量
print(sum(i.numel() for i in model.parameters()) / 10000)

#模型试算
out = model(**data)

out['loss'], out['logits'].shape

7695.616


(tensor(3.9203, grad_fn=<NllLossBackward0>), torch.Size([8, 49, 32128]))

In [7]:
#测试
def test():
    model.eval()

    #数据加载器
    loader_test = torch.utils.data.DataLoader(
        dataset=dataset['test'],
        batch_size=4,
        collate_fn=collate_fn,
        shuffle=True,
        drop_last=True,
    )

    for i, data in enumerate(loader_test):
        break
        
    #计算
    with torch.no_grad():
        out = model(**data)

    for i in range(4):
        input_ids = tokenizer.decode(data['input_ids'][i])
        pred = tokenizer.decode(out['logits'].argmax(dim=2)[i]),
        label = tokenizer.decode(data['decoder_input_ids'][i])

        #print('input_ids=', input_ids)
        print('pred=', pred)
        print('label=', label)
        print()
        
test()

pred= ('the citydefenderyle striker injured  is be  least two weeks  injury his mal  fitness  the  the the the the the the the the the the the the the the the',)
label= <pad> Plymouth Argyle striker Jimmy Spencer will miss at least three months after breaking and dislocating his ankle.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

pred= ('theley ares  che is in the top three of league   the season  says Pul Brom manager  Pulis  the  the the the the the the the the the the',)
label= <pad> Burnley's Sean Dyche is in the top three Premier League managers of the year, says West Brom boss Tony Pulis.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

pred= ('the in be be loans much as 3.$3 million £100m) in the assistance  the government government  the</s> the the the the the the the the the the the the the the',)
label= <pad> Farmers may soon receive as much as A$100m (£53m) in drought assistance from the Australian government.</s><pad><pad><pad><pad>

In [8]:
from transformers.trainer_pt_utils import get_parameter_names
from transformers import AdamW
from transformers.optimization import get_scheduler


#训练
def train():
    #一个工具函数,能够获取模型中所有参数的名字列表
    parameter_names = get_parameter_names(model, [torch.nn.LayerNorm])

    #定义哪些参数参与weight_decay
    parameter_names = [i for i in parameter_names if 'bias' not in i]

    #根据参数要参与weight_decay来把参数分为两组
    parameter_names = [
        {
            'params':
            [p for i, p in model.named_parameters() if i in parameter_names],
            'weight_decay':
            1e-2,
        },
        {
            'params': [
                p for i, p in model.named_parameters()
                if i not in parameter_names
            ],
            'weight_decay':
            0.0,
        },
    ]

    #定义优化器
    optimizer = AdamW(parameter_names, betas=(0.9, 0.999), eps=1e-8, lr=2e-5)

    #定义lr调整器
    scheduler = get_scheduler(name='linear',
                              num_warmup_steps=0,
                              num_training_steps=len(loader),
                              optimizer=optimizer)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.train()
    model.to(device)
    for i, data in enumerate(loader):
        for k in data.keys():
            data[k] = data[k].to(device)
            
        out = model(**data)
        loss = out['loss']

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        optimizer.zero_grad()
        model.zero_grad()

        if i % 50 == 0:
            pred = tokenizer.decode(out['logits'].argmax(dim=2)[0]),

            label = tokenizer.decode(data['decoder_input_ids'][0])

            lr = optimizer.state_dict()['param_groups'][0]['lr']
            print(i, loss.item(), lr)
            print('pred=', pred)
            print('label=', label)
            print()

    model.to('cpu')


if push_to_hub:
    train()
    model.push_to_hub(repo_id=repo_id, use_auth_token=hub_token)

/root/anaconda3/envs/pt39/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0 3.7945587635040283 1.9992e-05
pred= ('s are being  and  elephantt  of reasons.  are some  elephant them </s></s>              elephant elephant  elephant    ',)
label= <pad> Elephants are absolutely amazing animals for a number of reasons - here are just 11 of them.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

50 3.673530101776123 1.9592e-05
pred= ("Kiloughgiel led Closter 3s LoyanStartland 3. Killlysaannaa -12  -5  the's lastrim S Hur semiling Final  ygalle </s>",)
label= <pad> Loughgiel defeated Ulster champions and All-Ireland finalists Cushenhdall 1-15 to 1-12 in Sunday's Antrim Senior Club Hurling final in Ballycastle.

100 3.3030643463134766 1.9192000000000002e-05
pred= ('the of Man is have beenly voted thea new to the journalists in the Housex House. whichnwald.</s></s> the the the the the the the the the the the the the the the The',)
label= <pad> Isle of Man politicians have unanimously backed a move to allow c

1050 2.9269957542419434 1.1592000000000002e-05
pred= ('The are be  two and 50,000 people of slavery in the UK, the than the estimates. according has the number Office said.</s></s> The The The The The The The The',)
label= <pad> There could be between 10,000 and 13,000 victims of slavery in the UK, higher than previous figures, analysis for the Home Office suggests.</s><pad><pad><pad><pad><pad><pad><pad><pad>

1100 2.6950154304504395 1.1192e-05
pred= ('The annual will set be place in memorial Wommeoral Fal, honour the th anniversary of the British of the conflictklands conflict.</s>: The The Britain',)
label= <pad> An event is to take place at the Senedd in Cardiff to mark the 35th anniversary of the end of the Falklands War.</s><pad><pad><pad>

1150 2.7338650226593018 1.0792000000000001e-05
pred= ("Carl Grainger's header-half header for Carlisle United a - win over Carl beated in2-1. theage.</s></s> Steven Steven",)
label= <pad> Danny Grainger's second-half penalty helped Carlisle to 

2100 2.870434284210205 3.192e-06
pred= ('A group trial of Du of  with severeDM severe serious diseasedecular disease ( been at</s>. Bristol The Bristol The Bristol Bristol A Bristol A A Bristol A A A A A A A A A A The A A A A A The A A A A A A A A The A The A A',)
label= <pad> A clinical trial into treatment for children with a potentially fatal neuromuscular disorder has begun.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

2150 2.66827130317688 2.792e-06
pred= ('A poll have  for more to bereature"proofing and future of pollinators plants and which and insects to</s></s> The The The A A',)
label= <pad> International scientists are calling for action to "future-proof" the prosperity of pollinating insects, birds and mammals.</s><pad><pad><pad><pad><pad>

2200 2.7905874252319336 2.392e-06
pred= ('A Match one Phil Taylor Gerwen

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

In [9]:
#直接使用我训练好的模型
model = Model.from_pretrained(repo_id)

test()

pred= ('Thes have working to rebuild the canal canalsea canal Canal it of a project to rebuild the canal length of the canalway to to the. the.</s>',)
label= <pad> Volunteers are helping to restore the historic Swansea Canal as part of a project to bring the full length of the waterway back into use for boats.

pred= ('The inquiry into thea child of misconduct of misconduct hasinvolving children children ofs charity Kids Company has been launched. the National Police. which charity and said.</s></s> The',)
label= <pad> An investigation into a number of allegations of crime involving the children's charity Kids Company has been launched by the Metropolitan Police, the BBC has learned.</s><pad>

pred= ('Bristol Citydefenderyle haver Spencer Spencer has be the least two weeks of being the mal his squad after</s></s> Bristol Bristol Bristol Bristol Bristol Bristol Bristol Bristol Bristol Bristol',)
label= <pad> Plymouth Argyle striker Jimmy Spencer will miss at least three months after bre